In [ ]:
pip install -U langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 7.3 MB/s eta 0:00:00


In [ ]:
from typing import TypedDict, List, Union, Annotated, Sequence
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage, BaseMessage
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage],add_messages]

In [ ]:
@tool
def add(a: int, b:int):
    """This is an addition function that adds 2 numbers together"""

    return a + b

@tool
def subtract(a: int, b: int):
    """Subtraction function"""
    return a - b

@tool
def multiply(a: int, b: int):
    """Multiplication function"""
    return a * b

tools = [add, subtract, multiply]

In [ ]:
llm = ChatGroq(
    groq_api_key="",
    model_name="llama-3.1-8b-instant"
).bind_tools(tools)

In [ ]:
def model_call(state: AgentState) -> AgentState:
  system_prompt = SystemMessage(content=
        "You are my AI assistant, please answer my query to the best of your ability."
  )
  response = llm.invoke([system_prompt] + state["messages"])
  return {"messages": [response]}

def should_continue(state: AgentState):
  messages = state["messages"]
  last_message = messages[-1]
  if not last_message.tool_calls:
    return "end"
  else:
    return "continue"


In [ ]:
graph = StateGraph(AgentState)

graph.add_node("model_call",model_call)

tool_node = ToolNode(tools=tools)
graph.add_node("tools",tool_node)

graph.add_conditional_edges(
    "model_call",
    should_continue,
    {
        "continue": "tools",
        "end": END
    }
)

graph.add_edge(START, "model_call")
graph.add_edge("tools", "model_call")


agent=graph.compile()

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [ ]:
inputs = {"messages": [("user", "Add 40 + 12 and then multiply it with 9 ")]}
print_stream(agent.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Add 40 + 12 and then multiply it with 9 
================================== Ai Message ==================================
Tool Calls:
  add (vvk5vzmjt)
 Call ID: vvk5vzmjt
  Args:
    a: 40
    b: 12
  multiply (vtsb62tc2)
 Call ID: vtsb62tc2
  Args:
    a: 52
    b: 9
================================= Tool Message =================================
Name: multiply

468
================================== Ai Message ==================================

The result of the operations you requested is 468.
